Arctic sea ice forecasting using deep neural networks.

In [ ]:
import numpy as np
from pathlib import Path
import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional
import xarray as xr

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

Load and prepare data.

In [ ]:
# path
path_data = Path("/home/yangliu/MLexpo/data")
erai_path = path_data / "ERA-Interim"
oras_path = path_data / "ORAS4"

In [ ]:
start_year = 2000
end_year = 2017
# number of weeks per year
nweeks = 48
# load data
sic_xr = xr.load_dataset(erai_path / "sic_weekly_erai_1979_2017.nc")
sic_xr = sic_xr.sel(year=slice(start_year, end_year), latitude=slice(81, 63),
 longitude=slice(18, 59.5))
sic_xr

In [ ]:
# get mask from ORAS4 data
mask_ocean = xr.where(sic_xr['sic'][0,0,:,:] < 0, 1, 0)
mask_ocean.rename('mask')

In [ ]:
ohc_xr = xr.load_dataset(oras_path / "ohc_monthly_oras2erai_1978_2017.nc")
# the coordinate name is wrong for week, it should be month
ohc_xr = ohc_xr.rename({'week': 'month'})
# 1 more year inlucded to calculate the relative difference
ohc_xr = ohc_xr.sel(year=slice(start_year - 1, end_year), latitude=slice(81, 63),
 longitude=slice(18, 59.5))
ohc_xr

In [ ]:
# turn dataset into sequences
sic_seq = sic_xr.sic.values.reshape(-1, sic_xr.latitude.size, sic_xr.longitude.size)
ohc_seq_monthly = ohc_xr.OHC.values.reshape(-1, ohc_xr.latitude.size, ohc_xr.longitude.size)

In [ ]:
# masked area from -1 to 0
sic_seq[sic_seq < 0] = 0
# interpolation of ohc from monthly to weekly
ohc_diff_weekly = (ohc_seq_monthly[1:,:,:] - ohc_seq_monthly[:-1,:,:]) / 4
ohc_seq = np.zeros((sic_xr.year.size * nweeks, sic_xr.latitude.size, sic_xr.longitude.size),dtype=float)
# calculate relative difference, note that the first year data is edge case
for i in range(4):
    ohc_seq[3-i::4,:,:] = ohc_seq_monthly[12:,:,:] - i * ohc_diff_weekly[11:,:,:]

In [ ]:
def normalizer(x):
    max_value = np.amax(x)
    min_value = np.amin(x)
    y = (x - min_value)/(max_value - min_value)
    return y

In [ ]:
# normalize ohc
ohc_norm_seq = normalizer(ohc_seq)

Train/test data splitting.

In [ ]:
# we deal with timeseries, the data should not be shuffled
train_years = 12

Build neural network.

In [ ]:
class MetNet(nn.Module):
    def __init__(self, input_channels = 2, kernels=[2, 4, 1], dropout = 0.2):
        super().__init__()
        self.kernels = kernels
        # layer 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(input_channels, kernels[0], kernel_size=5, stride = 1, padding = 2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # layer 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(kernels[0], kernels[1], kernel_size=5, stride = 1, padding = 2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # layer 3
        self.layer3 = nn.Sequential(
            nn.Conv2d(kernels[1], kernels[2], kernel_size=5, stride = 1, padding = 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        return x

Set up the environment for training.

In [ ]:
print ("Pytorch version {}".format(torch.__version__))
#use_cuda = torch.cuda.is_available()
#print("Is CUDA available? {}".format(use_cuda))
input_channels = 2
epochs = 30
kernels = [2, 4, 1]
learning_rate = 0.01
dropout = 0.2

In [ ]:
# construct model
model = MetNet(input_channels, kernels, dropout)
# choose optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# loss function
loss_fn = torch.nn.MSELoss(reduction='mean')
# check the setups
print(model)
print(loss_fn)
print(optimizer)

In [ ]:
hist = np.zeros(epochs)
# switch model into train mode
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for timestep in range(train_years * nweeks):
        X_train = torch.tensor(np.stack((sic_seq[timestep,:,:],
                                        ohc_norm_seq[timestep,:,:]))
                              ).float().view(-1, input_channels, 
                                             sic_xr.latitude.size,
                                             sic_xr.longitude.size)
        x_var = torch.autograd.Variable(X_train)
        y_train = torch.tensor(sic_seq[timestep+1,:,:]).float().view(-1, 1, 
                                                                     sic_xr.latitude.size,
                                                                     sic_xr.longitude.size)
        y_var = torch.autograd.Variable(y_train)
        # forward pass
        outputs = model(x_var)
        # check loss
        loss = loss_fn(outputs, y_var)
        # zero the parameter gradients
        optimizer.zero_grad()
        # computing gradients
        loss.backward()
        # accumulating running loss
        running_loss += loss.item()
        # updated weights based on computed gradients
        optimizer.step()

    if epoch % 2 == 0:    
        print(f'Epoch {epoch + 1}/{epochs} running accumulative loss across all batches:' + 
              f'{running_loss:.3f}')
    running_loss = 0.0

In [ ]:
# save the general checkpoint
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item()
            }, Path('../xai/models/metnet_training_checkpoint.pt'))
print("The checkpoint of the model and training status is saved.")

# export to onnx format
onnx_file = Path('../xai/models/metnet.onnx')
# Export the model
torch.onnx.export(model,               # model being run
                  x_var,                         # model input (or a tuple for multiple inputs)
                  onnx_file,                 # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
print("The model is exported to onnx format.")

Model evaluation.

In [ ]:
timestep_test = train_years * nweeks + 5
# evalutation mode
model.eval()
X_test = torch.tensor(np.stack((sic_seq[timestep_test,:,:],
                                ohc_norm_seq[timestep_test,:,:]))
                        ).float().view(-1, input_channels, 
                                        sic_xr.latitude.size,
                                        sic_xr.longitude.size)
x_var = torch.autograd.Variable(X_test, requires_grad=False)
pred = model(x_var)

In [ ]:
plt.imshow(pred[0,0,:,:].data.numpy())
plt.colorbar()

In [ ]:
plt.imshow(sic_seq[timestep_test+1,:,:])
plt.colorbar()

Explainable AI.

In [ ]:
from captum.attr import IntegratedGradients, DeepLift, GradientShap

In [ ]:
ig = IntegratedGradients(model)
dl = DeepLift(model)
gs = GradientShap(model)

ig_attr_test = ig.attribute(X_test, n_steps=10)
dl_attr_test = dl.attribute(X_test)
gs_attr_test = gs.attribute(X_test, X_train)